     |████████████████████████████████| 92kB 4.0MB/s 
     |████████████████████████████████| 122kB 16.6MB/s 
  Created wheel for neat: filename=neat-0.4.1-cp36-none-any.whl size=8077 sha256=e0655540af3d8fdbc72894b7ecf437d9e0d5e58e179637836294556d39ecb1df
  Stored in directory: /root/.cache/pip/wheels/41/81/ca/a1b44c376b67d1b788dd0e2f92c967d7b44fec59eeec66667d
Successfully built neat
     |████████████████████████████████| 11.4MB 93kB/s 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pip install neat
!pip install pygame

import pygame, sys, time, random
import neat,os,pickle
from math import sqrt
os.environ['SDL_VIDEODRIVER']='dummy'

# Difficulty settings
# Easy      ->  10
# Medium    ->  25
# Hard      ->  40
# Harder    ->  60
# Impossible->  120
difficulty = 120
difficulty_real = 30

# Window size
frame_size_x = 720
frame_size_y = 480
pygame.font.init()
pygame.display.init()
# Checks for errors encountered
#check_errors =  pygame.init()#pygame.display.init()#
#print( pygame.display.list_modes() )
# pygame.init() example output -> (6, 0)
# second number in tuple gives number of errors
# if check_errors[1] > 0:
#     print(f'[!] Had {check_errors[1]} errors when initialising game, exiting...')
#     sys.exit(-1)
# else:
#     print('[+] Game successfully initialised')


# Initialise game window
pygame.display.set_caption('Snake Eater (name subject to change)')
game_window = pygame.display.set_mode((frame_size_x, frame_size_y))
resume= False
root="gdrive/My Drive/Colab Notebooks/coeus/"
restore_file="SnakeAiData"

# Colors (R, G, B)
black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
red = pygame.Color(255, 0, 0)
green = pygame.Color(0, 255, 0)
blue = pygame.Color(0, 0, 255)
# FPS (frames per second) controller
# fps_controller = pygame.time.Clock()
class snake:
# Game variables
    def __init__(self):#,snake_pos = [100, 50]):
        a=random.randrange(20,700,10)
        k=random.randrange(20,450,10)
        self.snake_pos=[a,k]
        self.snake_body = [[a, k], [a-10, k], [a-(2*10), k]]
        self.closeness=0
        self.snake_alive=True
        self.moves=500
        self.rand_food_pos()

    def rand_food_pos(self):
        self.food_pos = [random.randrange(1, (frame_size_x//10)) * 10, random.randrange(1, (frame_size_y//10)) * 10]
        self.food_spawn = True
        self.getstartdist()

    def getstartdist(self):
        self.distOrig=abs(self.food_pos[0]-self.snake_pos[0])+abs(self.food_pos[1]-self.snake_pos[1])
        self.closeness=0

    def getdistfit(self,ge):
            a=abs(self.food_pos[0]-self.snake_pos[0])+abs(self.food_pos[1]-self.snake_pos[1])
            if a<=self.distOrig:
                ge.fitness+=1
                self.closeness+=1
            else:
                ge.fitness-=2
                self.closeness-=2
            self.distOrig=a
    direction = 'RIGHT'
    change_to = direction

    def show_score(self,score,choice, color, font, size,string):
        score_font = pygame.font.SysFont(font, size)
        score_surface = score_font.render(string +' : ' + str(score), True, color)
        score_rect = score_surface.get_rect()
        if choice == 1:                             #fitness
            score_rect.midtop = (frame_size_x/10, 15)
        elif choice == 2:
            score_rect.midtop = (frame_size_x/2, frame_size_y/1.15) #moves
        elif choice == 0:
            score_rect.midtop = (frame_size_x- frame_size_x/10, 15) #closepoint
        game_window.blit(score_surface, score_rect)

    def get1dirinfo(self,x,y,xx,yy,fx,fy):
        global frame_size_x,frame_size_y
        while(x >= 0 and x <= frame_size_x and y >= 0 and y <= frame_size_y):
            x += xx
            y += yy
            if(x==fx and y==fy):
                return int(0)
            else:
                for sx,sy in enumerate(self.snake_body):
                    if x==sx and y==sy:
                        return int(1)
                else:
                    return int(2)

    def getdirinfo(self):
        dir = []
        # 0 food
        # 1 body
        # 2 boundary
        dir.append(self.get1dirinfo(self.snake_pos[0],self.snake_pos[1],0,-10,self.food_pos[0],self.food_pos[1]))#up
        dir.append(self.get1dirinfo(self.snake_pos[0],self.snake_pos[1],0,10,self.food_pos[0],self.food_pos[1]))#DOWN
        dir.append(self.get1dirinfo(self.snake_pos[0],self.snake_pos[1],-10,0,self.food_pos[0],self.food_pos[1]))#Left
        dir.append(self.get1dirinfo(self.snake_pos[0],self.snake_pos[1],10,0,self.food_pos[0],self.food_pos[1]))#RIGHT
        return dir


    def getquaterinfo(self,food,head):
        qs=[]
        if(head[0]>=food[0] and head[1]<=food[1]): #Q1
            qs.append(1)
        else:
            qs.append(0)

        if(head[0]>=food[0] and head[1]>=food[1]): #Q2
            qs.append(1)
        else:
            qs.append(0)

        if(head[0]<=food[0] and head[1]>=food[1]):#Q3
            qs.append(1)
        else:
            qs.append(0)

        if(head[0]<=food[0] and head[1]<=food[1]):#Q4
            qs.append(1)
        else:
            qs.append(0)
        return qs


    def give_arg(self):
        sx = self.snake_pos[0]
        sy = self.snake_pos[1]
        quaterinfo = self.getquaterinfo(self.food_pos,self.snake_pos)
        dirs = self.getdirinfo()
        return (self.snake_pos[0],self.snake_pos[1],self.food_pos[0],self.food_pos[1],quaterinfo[0],quaterinfo[1],quaterinfo[2],quaterinfo[3],dirs[0],dirs[1],dirs[2],dirs[3])

    def chkdirs(self,food):
        views = []
        x = self.snake_pos[0]
        y = self.snake_pos[1]
        views.append(self.chk(x,y,10,-10,food[0],food[1]))
        views.append(self.chk(x,y,10,10,food[0],food[1]))
        views.append(self.chk(x,y,-10,-10,food[0],food[1]))
        views.append(self.chk(x,y,-10,10,food[0],food[1]))
        views.append(self.chk(x,y,10,0,food[0],food[1]))
        views.append(self.chk(x,y,0,-10,food[0],food[1]))
        views.append(self.chk(x,y,0,10,food[0],food[1]))
        views.append(self.chk(x,y,-10,0,food[0],food[1]))
        return views
    def chk(self, x,y,xi,yi, fx,fy):
        global frame_size_x, frame_size_y
        val = []
        ini = [x,y]
        cbody = [x,y]
        food = sqrt(((ini[0] - fx)**2) + ((ini[1] - fy)**2))
        body = False

        while(x >= 0 and x < frame_size_x and y >= 0 and y < frame_size_y):
            x += xi
            y += yi
            t = sqrt(((ini[0] - fx)**2) + ((ini[1] - fy)**2))
            if(t < food):
                food = t
            if(not body):
                for i in range(1, len(self.snake_body)):
                    if((x == self.snake_body[i][0]) and (y == self.snake_body[i][1])):
                        cbody = [x,y]
                        body = True
                        break
        val.append(food)
        val.append(sqrt(((ini[0] - cbody[0])**2) + ((ini[1] - cbody[1])**2)))
        val.append(sqrt(((ini[0] - x)**2) + ((ini[1] - y)**2)))
        return val

def check_out_of_bounds(snakes,genomes,nets,i):
    if snakes[i].snake_pos[0] < 0 or snakes[i].snake_pos[0] > frame_size_x-10:
        # game_over
        genomes[i].fitness -= 2
        snakes[i].snake_alive = False
        snakes.pop(i)
        genomes.pop(i)
        nets.pop(i)
        return -1


    if snakes[i].snake_pos[1] < 0 or snakes[i].snake_pos[1] > frame_size_y-10:
        # game_over
        genomes[i].fitness -= 2
        snakes[i].snake_alive = False
        snakes.pop(i)
        genomes.pop(i)
        nets.pop(i)
        return -1

    # Touching the snake body
    for block in snakes[i].snake_body[1:]:
        if snakes[i].snake_pos[0] == block[0] and snakes[i].snake_pos[1] == block[1]:
            # game_over
            genomes[i].fitness -= 3
            snakes[i].snake_alive = False
            snakes.pop(i)
            genomes.pop(i)
            nets.pop(i)
            return -1
    if genomes[i].fitness>=500:
        snakes[i].moves=500
    elif genomes[i].fitness <=0:
        snakes[i].pop(i)
        genomes[i].pop(i)
        nets[i].pop(i)
        return -1
    return 0



def play_game(genomes, config):
    # Main logic

    fps = pygame.time.Clock()
    for _,ge in genomes:
        net = neat.nn.recurrent.RecurrentNetwork.create(ge, config)
        sn =snakes()
        sn.snake_alive = True
        while (sn.snake_alive and moves>0):
            fps.tick(difficulty)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

            args =  sn.give_arg()
            output = net.activate(args)
            sn.getdistfit(ge)
            if(output[0] == max(output) and not sn.direction == 'DOWN'):
                sn.snake_pos[1] -= 10
                sn.direction = 'UP'
            elif(output[1] == max(output) and not sn.direction == 'UP'):
                sn.snake_pos[1] += 10
                sn.direction = 'DOWN'
            elif(output[2] == max(output) and not sn.direction == 'RIGHT'):
                sn.snake_pos[0] -= 10
                sn.direction = 'LEFT'
            elif(output[3] == max(output) and not sn.direction == 'LEFT'):
                sn.snake_pos[0] += 10
                sn.direction = 'RIGHT'

            # Snake body growing mechanism
            sn.snake_body.insert(0, list(sn.snake_pos))
            if sn.snake_pos[0] == sn.food_pos[0] and sn.snake_pos[1] == sn.food_pos[1]:
                sn.food_spawn = False
            else:
                sn.snake_body.pop()

            # Spawning food on the screen
            if not sn.food_spawn:
                sn.rand_food_pos()
            # GFX
            game_window.fill(black)
            for pos in sn.snake_body:
                # Snake body
                # .draw.rect(play_surface, color, xy-coordinate)
                # xy-coordinate -> .Rect(x, y, size_x, size_y)
                pygame.draw.rect(game_window, green, pygame.Rect(pos[0], pos[1], 10, 10))
            # Snake food
            pygame.draw.rect(game_window, white, pygame.Rect(sn.food_pos[0], sn.food_pos[1], 10, 10))
            # Game Over conditions
            # Getting out of bounds
            if sn.snake_pos[0] < 0 or sn.snake_pos[0] > frame_size_x-10:
                # game_over
                sn.snake_alive = False
                break

            if sn.snake_pos[1] < 0 or sn.snake_pos[1] > frame_size_y-10:
                # game_over
                sn.snake_alive = False
                break

            # Touching the snake body
            for block in sn.snake_body[1:]:
                if sn.snake_pos[0] == block[0] and sn.snake_pos[1] == block[1]:
                    # game_over
                    sn.snake_alive = False
                    break

            pygame.display.update()
            fps.tick(difficulty_real)

def game(genome, config):
    # Main logic
    nets,snakes,genomes = [],[],[]
    fps = pygame.time.Clock()
    for _,ge in genome:
        sn = snake()
        snakes.append(sn)
        ge.fitness = 0
        genomes.append(ge)
        net = neat.nn.recurrent.RecurrentNetwork.create(ge, config)
        nets.append(net)

    while(snakes):
        #fps.tick(difficulty)
        print('k')
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        args=[]
        #ge.fitness += 0.01
        for i in range(len(snakes)):
            arg=snakes[i].give_arg()
            args.append(arg)

        for i in range(len(snakes)):
            output = nets[i].activate(args[i])
            snakes[i].getdistfit(genomes[i])
            if(output[0] == max(output) and not snakes[i].direction == 'DOWN'):
                snakes[i].snake_pos[1] -= 10
                snakes[i].moves -= 1
                snakes[i].direction = 'UP'
            elif(output[1] == max(output) and not snakes[i].direction == 'UP'):
                snakes[i].snake_pos[1] += 10
                snakes[i].moves -= 1
                snakes[i].direction = 'DOWN'
            elif(output[2] == max(output) and not snakes[i].direction == 'RIGHT'):
                snakes[i].snake_pos[0] -= 10
                snakes[i].moves -= 1
                snakes[i].direction = 'LEFT'
            elif(output[3] == max(output) and not snakes[i].direction == 'LEFT'):
                snakes[i].snake_pos[0] += 10
                snakes[i].moves -= 1
                snakes[i].direction = 'RIGHT'

        # Snake body growing mechanism
        for i in range(len(snakes)):
            snakes[i].snake_body.insert(0,list(snakes[i].snake_pos))
            if snakes[i].snake_pos[0] == snakes[i].food_pos[0] and snakes[i].snake_pos[1] == snakes[i].food_pos[1]:
                genomes[i].fitness += 100
                snakes[i].food_spawn = False
            else:
                snakes[i].snake_body.pop()

            # Spawning food on the screen
            if not snakes[i].food_spawn:
                snakes[i].rand_food_pos()
                snakes[i].getstartdist()
            # GFX
            for pos in snakes[i].snake_body:
                # Snake body
                # .draw.rect(play_surface, color, xy-coordinate)
                # xy-coordinate -> .Rect(x, y, size_x, size_y)
                pygame.draw.rect(game_window, green, pygame.Rect(pos[0], pos[1], 10, 10))
            # Snake food
            pygame.draw.rect(game_window, white, pygame.Rect(snakes[i].food_pos[0], snakes[i].food_pos[1], 10, 10))
            # Game Over conditions
            # Getting out of bounds
            i=0
            while(True):
                if(0<len(snakes)):
                    i+=check_out_of_bounds(snakes,genomes,nets,i)
                ++i
        # Refresh game screen
        pygame.display.update()
        # Refresh rate
        fps.tick(difficulty)

def gg(conf_f):
    conf = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            conf_f)
    if resume == True:
        p = neat.Checkpointer.restore_checkpoint(restore_file)
    else:
        p = neat.Population(conf)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(1))
    winner = p.run(game, 200)
    with open(root+'winner.pkl', 'wb') as output:
        pickle.dump(winner, output, 1)

    drive.flush_and_unmount()
if __name__ == '__main__':
    conf_file = root+'conf.gg'
    gg(conf_file)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


AttributeError: ignored

In [0]:
import neat
neat.population.Population()


AttributeError: ignored

In [0]:
|